# Ares Demo

Example on running Ares.

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ares import construct

## Quickstart (simple)

If `ares` is installed and the config file is valid, simply run the command

```sh
$ python -m ares /path/to/config.json
```

This will run simulation and automatically output the commonly used results.

In [ ]:
! cd .. && python -m ares configs/projected_gradient_descent.json

## Quickstart (intermediate)

For more choice on running the simulation, the environment needs to be constructed and run manually. The following steps will guide through the process.

### Config File

First, load the configuration for the simulation. You can either read the config file from a JSON file or manually define the dictionary in Python. Ensure the path to the defense models and checkpoints are correct.

In [ ]:
config = {
    "attacker": {
        "attacks": [
            {
                "type": "evasion",
                "name": "ProjectedGradientDescent",
                "params": {
                    "norm": "inf",
                    "eps": 0.03137254901,
                    "eps_step": 0.00784313725,
                    "num_random_init": 0,
                    "max_iter": 1,
                    "targeted": False,
                    "batch_size": 1,
                    "verbose": False
                }
            }
        ]
    },
    "defender": {
        "probabilities": [0.3, 0.7],
        "models": [
            {
                "file": "../data/models/resnet.py",
                "name": "resnet18_nat",
                "params": {
                    "num_classes": 10
                },
                "checkpoint": "../data/state_dicts/resnet18_nat.pth"
            },
            {
                "file": "../data/models/resnet.py",
                "name": "resnet18_adv",
                "params": {
                    "num_classes": 10
                },
                "checkpoint": "../data/state_dicts/resnet18_adv.pth"
            }
        ]
    },
    "scenario": {
        "threat_model": "whitebox",
        "num_episodes": 50,
        "max_rounds": 50,
        "dataset": {
            "name": "cifar10",
            "input_shape": [3, 32, 32],
            "num_classes": 10,
            "clip_values": [0, 1],
            "random_noise": True,
            "dataroot": "../data/cifar10"
        }
    }
}

### Construct Environment

Next, create the simulation environment. A helper function `construct(config)` is provided which will automatically create the attacker, defender, and scenario components based on the config file. This will initialize the `gym` environment which can be run.

In [ ]:
# create environment
env = construct(config)
env

### Run Experiment

Once the environment is created, run the simulation for the number of episodes desired. This is always specified in the scenario component. Record the number of rounds and queries until the attacker wins for each trial or the defender wins by a timeout.

In [ ]:
episode_rewards = []
episode_queries = []
for episode in range(env.scenario.num_episodes):
    print(f"=== Episode {episode + 1} ===")

    # initialize environment
    observation = env.reset()
    x = observation["x"]
    y = observation["y"]
    done = False

    # run simulation
    while not done:
        action = {
            "x": x,
            "y": y,
        }
        observation, reward, done, info = env.step(action)
        defense = observation["defense"]
        attack = observation["attack"]
        x = observation["x_adv"]
        y_pred = observation["y_pred"]
        eps = observation["eps"]
        evaded = observation["evaded"]
        detected = observation["detected"]
        winner = observation["winner"]
        step_count = info["step_count"]
        queries = info["queries"]

        print(f"Round {step_count:2}: defense = {defense}, attack = {attack}")
        print(f"\t [label = {y[0]} | pred = {y_pred[0]}], eps = {eps:.6f}, queries = {queries}")
        if evaded:
            print("\t attacker evaded")
        elif detected:
            print("\t attacker was detected")

        print(f"Game end: {winner} wins after {reward} rounds and {queries} queries")
        episode_rewards.append(reward)
        episode_queries.append(queries)

### Calculate Statistics

Finally, calculate the statistics and create the visualizations for the experiment.

In [ ]:
reward_mean = np.mean(episode_rewards)
reward_stddev = np.std(episode_rewards)
reward_median = np.median(episode_rewards)
print(f"Rounds:  mean = {reward_mean}, stddev = {reward_stddev:.3f}, median = {reward_median}")

queries_mean = np.mean(episode_queries)
queries_stddev = np.std(episode_queries)
queries_median = np.median(episode_queries)
print(f"Queries: mean = {queries_mean}, stddev = {queries_stddev:.3f}, median = {queries_median}")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax[0].hist(episode_rewards)
ax[0].set(title='Distribution of Episode Rounds', xlabel='Rounds', ylabel='Count')
ax[1].hist(episode_queries)
ax[1].set(title='Distribution of Episode Queries', xlabel='Queries', ylabel='Count')
plt.show()

## Quickstart (advanced)

For full control over the entire simulation, each of the components for the simulation environment can be constructed manually. This is not recommended unless full control is needed.